# Import and preparation

In [3]:
import sys

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# import torch
# import torch.nn as nn
# from torch import optim
# import torch.nn.functional as F

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
SQUAD_DATASET_PATH = 'Datasets/SQuAD/v2.0/train-v2.0-translated.json'

df_squad = pd.read_json(SQUAD_DATASET_PATH)
df_squad = df_squad
print(df_squad.shape)
print(df_squad)

df_squad_original = pd.read_json('Datasets/SQuAD/v2.0/train-v2.0.json')
print(df_squad_original.shape)
df_squad_original.head()

(442, 2)
                                           title  \
0                                        Beyonce   
1                                Frédéric Chopin   
2    Hubungan Tiongkok-Tibet selama dinasti Ming   
3                                           IPod   
4         The Legend of Zelda: Twilight Princess   
..                                           ...   
437                                      Infeksi   
438                                      Berburu   
439                                    Kathmandu   
440                               Infark miokard   
441                                      Masalah   

                                            paragraphs  
0    [{'qas': [{'question': 'Kapan Beyonce mulai me...  
1    [{'qas': [{'question': 'Bagaimana kewarganegar...  
2    [{'qas': [{'question': 'Siapakah Wang Jiawei d...  
3    [{'qas': [{'question': 'Perusahaan mana yang m...  
4    [{'qas': [{'question': 'Apa kategori game Lege...  
..                      

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...


In [5]:
df_squad_original['title'] = df_squad_original['data'].apply(lambda x: x.get('title'))
df_squad_original['paragraphs'] = df_squad_original['data'].apply(lambda x: x.get('paragraphs'))
df_squad_original.drop(columns=['data'], inplace=True)

print(df_squad_original.iloc[220]['paragraphs'][0]['context'])
df_squad_original.iloc[220]['paragraphs'][0]['qas']

Gothic architecture is a style of architecture that flourished during the high and late medieval period. It evolved from Romanesque architecture and was succeeded by Renaissance architecture. Originating in 12th-century France and lasting into the 16th century, Gothic architecture was known during the period as Opus Francigenum ("French work") with the term Gothic first appearing during the later part of the Renaissance. Its characteristics include the pointed arch, the ribbed vault and the flying buttress. Gothic architecture is most familiar as the architecture of many of the great cathedrals, abbeys and churches of Europe. It is also the architecture of many castles, palaces, town halls, guild halls, universities and to a less prominent extent, private dwellings, such as dorms and rooms.


[{'question': 'What is one time period in which Gothic architecture flourished?',
  'id': '57268fad5951b619008f769b',
  'answers': [{'text': 'late medieval period', 'answer_start': 83}],
  'is_impossible': False},
 {'question': 'What style of architecture did Gothic architecture evolve from?',
  'id': '57268fad5951b619008f769c',
  'answers': [{'text': 'Romanesque architecture', 'answer_start': 121}],
  'is_impossible': False},
 {'question': 'What style of architecture came after the Gothic style?',
  'id': '57268fad5951b619008f769d',
  'answers': [{'text': 'Renaissance architecture', 'answer_start': 166}],
  'is_impossible': False},
 {'question': 'Where did the Gothic architecture style originate?',
  'id': '57268fad5951b619008f769e',
  'answers': [{'text': 'France', 'answer_start': 220}],
  'is_impossible': False},
 {'question': 'Gothic architecture is known for being commonly used in cathedrals and churches, what is one example of a lesser known type of structure in which Gothic arch

# Helper Functions

In [3]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return f'{m}m {s}s'


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return f'{asMinutes(s)} (- {asMinutes(rs)})'

# Data preprocessing

In [188]:
# for topic_idx in df_squad.iloc[0]['paragraphs']:
for question in df_squad.iloc[1]['paragraphs'][6]['qas']:
    answers = question['answers'][0]
    print(answers['text'])
#     # 0,0
#     if answers['text'] == 'Berbahaya dalam Cinta':
#         answers['text'] = 'Dangerously in Love'
#     elif answers['text'] == 'bernyanyi dan menari':
#         answers['text'] = 'menyanyi dan menari'
#     # 0,1
#     if answers['text'] == 'Beyonce':
#         answers['text'] = 'Beyoncé '
#     # 0,3
#     if answers['text'] == 'Metodis':
#         answers['text'] = 'Metodhis'
#     elif answers['text'] == 'Amerika Afrika':
#         answers['text'] = 'Afrika-Amerika'
#     # 1,0
    if answers['text'] == 'hanya perlahan':
        answers['text'] = 'piano solo'
#     # 1,3
    if answers['text'] == 'rencana':
        answers['text'] = 'piano'
    print(answers['text'])
    print()

Oktober
Oktober

Perancis
Perancis

alasan Istana
alasan Istana

seruling dan biola
seruling dan biola

penyakit
penyakit

Oktober 1810
Oktober 1810

Lyceum Warsawa
Lyceum Warsawa

seruling dan biola
seruling dan biola

rencana
piano

enam bulan
enam bulan

Perancis
Perancis

seruling dan biola
seruling dan biola

rencana
piano

Istana Saxon.
Istana Saxon.



In [263]:
import string

FIND_STRING_APPROX_ATTEMPT_LIMIT = 3
# SHORT_ANSWER_THRESHOLD = 3
SIMILARITY_THRESHOLD = 0.5
NOT_FOUND_VALUE = (-1, -1)

def remove_punctuations(text):
    table = str.maketrans('', '', string.punctuation)
    filtered = [w.translate(table) for w in text.split()]
    return ' '.join(filtered)

def find_word_in_list(list_word, word, start_search_idx=0):
    found = False
    i = start_search_idx
    while i < len(list_word) and not found:
        found = list_word[i] == word
        i += 1
    if found:
        return i-1
    else:
        return -1

def find_string_approx(context, answer, ignore_case_punc=False, f_out=sys.stdout):
    original_context_words = context.split()
    original_answer_words = answer.split()
    if ignore_case_punc:
        context_words = remove_punctuations(context).lower().split()
        answer_words = remove_punctuations(answer).lower().split()
    else:
        context_words = original_context_words
        answer_words = original_answer_words
    context_words_length = len(context_words)
    answer_words_length = len(answer_words)
    
#     if answer_words_length < SHORT_ANSWER_THRESHOLD:
#         char_pos_start = context.find(answer)
#         char_pos_end = char_pos_start + len(answer)
#         return (char_pos_start, char_pos_end)
    
    attempt = 0
    
    i_initial = -1
    j_initial = 0    
    i_start = i_initial
    j_start = j_initial
    while attempt < FIND_STRING_APPROX_ATTEMPT_LIMIT:
        if attempt > 0: print(f'Trying another starting word. Attempt {attempt+1}..', file=f_out)
        while i_start == i_initial and j_start < answer_words_length:
            i_start = find_word_in_list(context_words, answer_words[j_start], start_search_idx=i_initial+1)
            if i_start == i_initial:
                j_start += 1
        if i_start == -1:
            return NOT_FOUND_VALUE

        match_count = 0
        i = i_start
        j = j_start
        while i < context_words_length and j < answer_words_length:
            if context_words[i] == answer_words[j]:
                match_count += 1
                i += 1
                j += 1
            else:
                if j < answer_words_length-1 and context_words[i] == answer_words[j+1]:
                    match_count += 1
                    i += 1
                    j += 2
                elif i < context_words_length-1 and context_words[i+1] == answer_words[j]:
                    match_count += 1
                    i += 2
                    j += 1
                else:
                    i += 1
                    j += 1
        similarity = match_count / answer_words_length
        if similarity >= SIMILARITY_THRESHOLD:
            char_pos_start = 0
            for k in range(i_start):
                char_pos_start += len(original_context_words[k]) + 1
            char_pos_end = 0
            for k in range(i_start, i):
                char_pos_end += len(original_context_words[k]) + 1
            print(f'start:({i_start},{j_start}), end:({i},{j}), ({context_words[i_start]},{answer_words[j_start]}), {similarity}', file=f_out)
            return (char_pos_start, char_pos_start+char_pos_end-1)
        else:
            attempt += 1
            print('Not found, ', end='', file=f_out)
            i_initial = i_start    
    if attempt >= FIND_STRING_APPROX_ATTEMPT_LIMIT:
        print(f'({i},{j}): {match_count}/{answer_words_length}={similarity}', file=f_out)
        return NOT_FOUND_VALUE

In [264]:
find_string_approx('at, republik republik republik kepentingan dalam konflik atau dengan para pejuang, seperti Denmark-Norwegia. Republik Belanda, sekutu lama Inggris, mempertahankan kenetralannya, takut akan kemungkinan melawan Inggris dan Prusia melawan kekuatan besar Eropa, bahkan beru', 'Republik Belanda, sekutu lama Inggris, mempertahankan netralitasnya', True)
# find_string_approx('Śuddhodana bertekad untuk melihat putranya menjadi raja, jadi dia mencegahnya meninggalkan pekarangan istana. Tetapi pada usia 29, terlepas dari upaya ayahnya, Gautama berkelana ke luar istana beberapa kali. Dalam serangkaian pertemuan — yang dikenal dalam literatur Buddhis sebagai empat pemandangan — ia belajar tentang penderitaan orang-orang biasa, bertemu dengan seorang lelaki tua, seorang lelaki sakit, sebuah mayat, dan, akhirnya, seorang lelaki suci pertapa, yang tampaknya puas dan damai dengan Dunia. Pengalaman-pengalaman ini mendorong Gautama untuk meninggalkan kehidupan kerajaan dan melakukan pencarian spiritual.', 'kepala suku terpilih', True)

Not found, Trying another starting word. Attempt 2..
Not found, Trying another starting word. Attempt 3..
Not found, (10,7): 1/7=0.14285714285714285


(-1, -1)

In [253]:
ATTEMPT_LIMIT = 5
LEFT_OFFSET = 50
RIGHT_OFFSET = 80
OFFSET_SEEKING_ADDITIONAL = 15

IMPOSSIBLE_LOC = (-2, -2)

def get_answer_loc_new(context, answer_text, answer_loc, f_out=sys.stdout):
    if answer_text == '':
        return IMPOSSIBLE_LOC
    
    found = False
    attempt = 0
    more_offset = 0
    
    answer_text_total_words = len(answer_text.split())
            
    while attempt < ATTEMPT_LIMIT and not found:
        context_loc = (max(answer_loc[0]-LEFT_OFFSET-more_offset, 0), min(answer_loc[0]+len(answer_text)+RIGHT_OFFSET+more_offset, len(context)))
        prepared_context = context[context_loc[0]:context_loc[1]].lower()
        prepared_answer_text = answer_text.lower()
        
        answer_loc_start, answer_loc_end = find_string_approx(prepared_context, prepared_answer_text, ignore_case_punc=True, f_out=f_out)
        if answer_loc_start != -1:
            answer_loc_new = (
                answer_loc_start + context_loc[0],
                answer_loc_end + context_loc[0]
            )
            print(f'Indonesia answer_loc_new: ', end='', file=f_out)
            found = True
        else:
            attempt += 1
            more_offset += OFFSET_SEEKING_ADDITIONAL
    if attempt >= ATTEMPT_LIMIT:
        print(context[context_loc[0]:context_loc[1]], file=f_out)
        print('INDONESIA ANSWER_LOC NOT FOUND: ', end='', file=f_out)
        answer_loc_new = (-1, -1)
    print(answer_loc_new, file=f_out)
    return answer_loc_new

In [250]:
def calculate_add_context_failures(taken_topic_idx, taken_context_idx, f_out=sys.stdout):
    qas = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas']
    context = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['context']
    print(context, end='\n\n', file=f_out)

    failure_count = 0
    i = 0
    for question in qas:
        print(i, file=f_out)
        answers = question['answers']
        if len(answers) > 0:
            answer_text = answers[0]['text']
            answer_loc = int(answers[0]['answer_start'])
            answer_loc = (min(answer_loc, len(context) - 1), min(answer_loc + len(answer_text), len(context)))
        else:
            answer_text = ''
            answer_loc = ()
        print(f'English answer_loc: {answer_loc}', file=f_out)

        answer_loc_new = get_answer_loc_new(context, answer_text, answer_loc, f_out)
    
        # If not found, try the original english answer. Probably GoogleTranslate had translated name entities.
        if answer_loc_new == (-1, -1):
            answer_text = df_squad_original.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas'][i]['answers'][0]['text']
            answer_loc_new = get_answer_loc_new(context, answer_text, answer_loc, f_out)

        failure_count += 1 if answer_loc_new == (-1, -1) else 0
        
        i += 1
        if answer_loc_new == IMPOSSIBLE_LOC: print("Impossible question", file=f_out)
#         if answer_loc_new == (-1, -1):
        print(f'Question: {question["question"]}', file=f_out)
        print(f'Answer: {answers}', file=f_out)
        print(f'Located in context: {context[answer_loc_new[0]:answer_loc_new[1]]}', file=f_out)
        print('', file=f_out)
    print(f'Failure counts:{failure_count}', file=f_out)
    df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['failure_count'] = failure_count

In [247]:
for taken_topic_idx in range(df_squad.shape[0]):
    print(f'Topic: {taken_topic_idx}')
    for taken_context_idx in range(len(df_squad.iloc[taken_topic_idx]['paragraphs'])):
        print(f'\t{taken_context_idx}')
        calculate_add_context_failures(taken_topic_idx, taken_context_idx)

Topic: 0
	0
	1
	2
	3
	4
	5
	6
	7
	8
	9
	10
	11
	12
	13
	14
	15
	16
	17
	18
	19
	20
	21
	22
	23
	24
	25
	26
	27
	28
	29
	30
	31
	32
	33
	34
	35
	36
	37
	38
	39
	40
	41
	42
	43
	44
	45
	46
	47
	48
	49
	50
	51
	52
	53
	54
	55
	56
	57
	58
	59
	60
	61
	62
	63
	64
	65
Topic: 1
	0
	1
	2
	3
	4
	5
	6
	7
	8
	9
	10
	11
	12
	13
	14
	15
	16
	17
	18
	19
	20
	21
	22
	23
	24
	25
	26
	27
	28
	29
	30
	31
	32
	33
	34
	35
	36
	37
	38
	39
	40
	41
	42
	43
	44
	45
	46
	47
	48
	49
	50
	51
	52
	53
	54
	55
	56
	57
	58
	59
	60
	61
	62
	63
	64
	65
	66
	67
	68
	69
	70
	71
	72
	73
	74
	75
	76
	77
	78
	79
	80
	81
Topic: 2
	0
	1
	2
	3
	4
	5
	6
	7
	8
	9
	10
	11
	12
	13
	14
	15
	16
	17
	18
	19
	20
	21
	22
	23
	24
	25
	26
	27
	28
	29
	30
	31
	32
	33
	34
	35
	36
	37
	38
	39
	40
	41
	42
	43
	44
	45
	46
	47
	48
	49
	50
	51
	52
	53
	54
	55
	56
	57
	58
	59
	60
	61
	62
	63
	64
	65
	66
	67
	68
	69
	70
	71
Topic: 3
	0
	1
	2
	3
	4
	5
	6
	7
	8
	9
	10
	11
	12
	13
	14
	15
	16
	17
	18
	19
	20
	21
	22
	23
	24
	25
	26
	27
	28
	29
	30


# Add failure_percentage column

In [266]:
failure_percentage = [0.0 for _ in range(df_squad.shape[0])]
total_questions = [0 for _ in range(df_squad.shape[0])]

for taken_topic_idx in range(df_squad.shape[0]):
    print(f'Topic: {taken_topic_idx}')
    ctx_total_question = []
    failure_list = []
    
    for taken_context_idx in range(len(df_squad.iloc[taken_topic_idx]['paragraphs'])):
        print(f'\tContext_idx: {taken_context_idx}')
        context = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]
        qas_total_question = len(context['qas'])
        ctx_failure_percentage = context['failure_count']/qas_total_question
        print(f'\tFailures: {context["failure_count"]}/{qas_total_question} = {ctx_failure_percentage * 100:.2f}%')
        ctx_total_question.append(qas_total_question)
        failure_list.append(ctx_failure_percentage)
        
    total_questions[taken_topic_idx] = sum(ctx_total_question)
    failure_percentage[taken_topic_idx] = sum(failure_list)/len(failure_list)
    
df_squad['failure_percentage'] = failure_percentage
df_squad['total_questions'] = total_questions

Topic: 0
	Context_idx: 0
	Failures: 0/15 = 0.00%
	Context_idx: 1
	Failures: 0/12 = 0.00%
	Context_idx: 2
	Failures: 0/12 = 0.00%
	Context_idx: 3
	Failures: 2/13 = 15.38%
	Context_idx: 4
	Failures: 0/11 = 0.00%
	Context_idx: 5
	Failures: 1/13 = 7.69%
	Context_idx: 6
	Failures: 0/13 = 0.00%
	Context_idx: 7
	Failures: 0/10 = 0.00%
	Context_idx: 8
	Failures: 0/11 = 0.00%
	Context_idx: 9
	Failures: 0/15 = 0.00%
	Context_idx: 10
	Failures: 0/14 = 0.00%
	Context_idx: 11
	Failures: 0/10 = 0.00%
	Context_idx: 12
	Failures: 0/11 = 0.00%
	Context_idx: 13
	Failures: 0/13 = 0.00%
	Context_idx: 14
	Failures: 0/14 = 0.00%
	Context_idx: 15
	Failures: 0/13 = 0.00%
	Context_idx: 16
	Failures: 3/14 = 21.43%
	Context_idx: 17
	Failures: 1/11 = 9.09%
	Context_idx: 18
	Failures: 0/13 = 0.00%
	Context_idx: 19
	Failures: 1/13 = 7.69%
	Context_idx: 20
	Failures: 0/13 = 0.00%
	Context_idx: 21
	Failures: 0/12 = 0.00%
	Context_idx: 22
	Failures: 1/13 = 7.69%
	Context_idx: 23
	Failures: 0/10 = 0.00%
	Context_idx: 2

In [34]:
pd.set_option('display.max_rows', 50)

In [267]:
# df_analysis = df_squad[df_squad.failure_percentage > 0.12].drop(columns=['paragraphs'])
df_analysis = df_squad.drop(columns=['paragraphs'])
print(df_analysis.failure_percentage.describe())
df_analysis['failed_questions'] = df_analysis['failure_percentage'] * df_analysis['total_questions']
df_analysis = df_analysis.sort_values(by=['failed_questions'], ascending=False)
sorted_index = df_analysis.index
print(sorted_index.tolist())
print(df_analysis)
print(f"Dropped: {df_analysis['failed_questions'].sum()}")
print(f"Total: {df_analysis['total_questions'].sum()}")

count    442.000000
mean       0.043743
std        0.029264
min        0.000000
25%        0.024501
50%        0.036825
75%        0.057123
max        0.193571
Name: failure_percentage, dtype: float64
[11, 91, 89, 413, 151, 285, 174, 13, 12, 307, 381, 75, 438, 1, 308, 0, 171, 367, 262, 7, 284, 406, 10, 78, 378, 72, 178, 424, 360, 422, 349, 43, 357, 74, 81, 61, 373, 158, 329, 421, 305, 416, 361, 425, 267, 313, 161, 55, 229, 227, 399, 186, 14, 145, 379, 228, 163, 408, 210, 121, 71, 255, 97, 382, 194, 40, 214, 411, 238, 191, 47, 4, 225, 185, 347, 170, 76, 204, 220, 233, 412, 394, 252, 8, 206, 5, 44, 94, 248, 141, 116, 271, 340, 433, 107, 66, 300, 258, 232, 254, 152, 209, 123, 168, 90, 166, 150, 404, 344, 250, 407, 439, 33, 415, 144, 342, 6, 409, 231, 391, 207, 164, 169, 259, 60, 242, 243, 396, 293, 291, 162, 369, 193, 146, 218, 159, 73, 217, 403, 237, 173, 386, 95, 352, 80, 3, 437, 423, 147, 336, 377, 70, 100, 375, 309, 316, 351, 184, 86, 430, 46, 364, 337, 226, 197, 387, 127, 157, 345, 3

In [265]:
with open('answer_findings_log.txt', 'w', encoding='utf-8') as f_out:
    for i in range(len(sorted_index)):
        taken_topic_idx = sorted_index[i]
        print(f'Topic {taken_topic_idx}: {df_squad.iloc[taken_topic_idx]["title"]}', file=f_out)
        print(f'Rank: {i}', file=f_out)
        print(f'Failure rate: {df_analysis["failed_questions"][taken_topic_idx]:.0f}/{df_analysis["total_questions"][taken_topic_idx]} = {df_analysis["failure_percentage"][taken_topic_idx]:.2f}%', file=f_out)
        for taken_context_idx in range(len(df_squad.iloc[taken_topic_idx]['paragraphs'])):
            calculate_add_context_failures(taken_topic_idx, taken_context_idx, f_out)
            print('---------- END OF CONTEXT ----------', file=f_out)
        print(f'========== END OF TOPIC {taken_topic_idx} ==========', end='\n\n', file=f_out)

# Sequence to Sequence

## Encoder

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Decoder

In [10]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Attention

In [12]:
MAX_LENGTH = 10

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Training

In [15]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

# Plotting Results

In [17]:
plt.switch_backend('agg')
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# Evaluation

In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

# Train & Evaluate

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)